In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()


from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

# Initialize the Groq 

llm=ChatGroq(
    model="Llama3-8b-8192",
    groq_api_key=groq_api_key,
)

llm 

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000291A9C42B30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000291A9C40610>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
# !pip install beautifulsoup4
import warnings
warnings.filterwarnings("ignore")

In [6]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

In [7]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
#from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [13]:
import bs4
loader= WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title","post-header"),
        )
    ),
)

docs=loader.load()

len(docs)
#docs

1

# Divide the documents into smaller chunks

In [15]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

splits=text_splitter.split_documents(docs)

vector_store=Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
)

retriever=vector_store.as_retriever()

retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002918487A2C0>, search_kwargs={})

# Prompt template

In [16]:
system_prompt=(
    "You are an assistant for question answering tasks"
    "Use the following pieces of retrieved context to answer"
    "the question.If you dont know the answer, just say you dont know."
    "Use three sentences or less to answer the question."
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [17]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)


✅ Summary
You're creating a system where:

A retriever finds the best documents,

They get stuffed into a prompt with the question,

Then a language model generates the answer from that prompt.

In [20]:
response=rag_chain.invoke({"input":"What is Self-Reflection"})
response

{'input': 'What is Self-Reflection',
 'context': [Document(id='53b0e195-00e8-4ef6-84f2-e0f9ecbd2204', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Self-Reflection#\nSelf-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.\nReAct (Yao et al. 2023) integrates reasoning and acting within LLM by extending the action space to be a combination of task-specific discrete actions and the language space. The former enables LLM to interact with the environment (e.g. use Wikipedia search API), while the latter prompting LLM to generate reasoning traces in natural language.\nThe ReAct prompt template incorporates explicit steps for LLM to think, roughly formatted as:\nThought: ...\nAction: ...\nObservation: ...\n... (Repeated many times)'),
  Document(id='1e8e9025-b0af-4f32-b0

In [25]:
response=rag_chain.invoke({"input":"What is Self-Reflection"})
response["answer"]

'Self-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'

In [ ]:
# Here is doesnot have any chat history
# here we can able to see some irrelevant answers,which is not at all related to Self-Reflection
response=rag_chain.invoke({"input":"How can we achieve it"})
response["answer"]

'To achieve efficiency improvement, stability improvement of LLM outputs and external model services, we can try the following:\n\n1. Optimize LLM inference rounds.\n2. Improve communication with other models by reducing the long context window.'

# Adding Chat History

In [28]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder



contextualize_q_system_prompt =(
"Given a chat history and the latest user question "
"which might reference context in the chat history,"
"formulate a standalone question which can be understood "
"without the chat history. Do NOT answer the question, "
"just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
       ("human","{input}")
    ]
)

In [29]:
history_aware_retriever=create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt
)

history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002918487A2C0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [33]:
qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
       ("human","{input}")
    ]
)

In [34]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)

rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [39]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is Self-Reflection"
response=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response["answer"])
    ]
)

question2="Tell me about it again"
response2=rag_chain.invoke({"input":question2,"chat_history":chat_history})

print(response2["answer"])



Self-Reflection is a mechanism that helps agents learn from their past actions and decisions, allowing them to refine their plans and correct mistakes.


In [38]:
chat_history

[HumanMessage(content='What is Self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Self-Reflection is a process that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes.', additional_kwargs={}, response_metadata={})]